In [1]:
import torch
import pyvista

from torchfem import Solid
from torchfem.mesh import cube_hexa
from torchfem.materials import IsotropicElasticity3D, IsotropicKirchhoff3D

# Set default data type to double precision
torch.set_default_dtype(torch.float64)

# Generate beam
L = 5.0
P = -5
nodes, elements = cube_hexa(21, 5, 5, L, 1.0, 1.0)
tip = (nodes[:, 0] == L) & (nodes[:, 1] == 0.5) & (nodes[:, 2] == 0.5)

In [2]:
# Create model with small strain isotropic elasticity
material = IsotropicElasticity3D(E=1000.0, nu=0.3)
beam1 = Solid(nodes, elements, material)

# Assign boundary conditions
beam1.constraints[nodes[:, 0] == 0, :] = True
beam1.forces[tip, 2] = P

# Solve
u1, f1, σ1, F1, α1 = beam1.solve()

print("Tip deflection (ABAQUS reference):")
print(f"    X: {u1[tip,0].item():.3f} ( 0.000)")
print(f"    Y: {u1[tip,1].item():.3f} ( 0.000)")
print(f"    Z: {u1[tip,2].item():.3f} (-2.549)")
print(f"Maximum stress σ_xx: {σ1[:,0,0].max():.3f} (112.96)")
print(f"Maximum stress σ_xx: {σ1[:,0,0].min():.3f} (-112.96)")

Tip deflection (ABAQUS reference):
    X: -0.000 ( 0.000)
    Y: -0.000 ( 0.000)
    Z: -2.466 (-2.549)
Maximum stress σ_xx: 108.787 (112.96)
Maximum stress σ_xx: -108.787 (-112.96)


In [3]:
# Create model with Kirchhoff material
material = IsotropicKirchhoff3D(E=1000.0, nu=0.3)
beam2 = Solid(nodes, elements, material)

# Assign boundary conditions
beam2.constraints[nodes[:, 0] == 0, :] = True
beam2.forces[tip, 2] = P

# Solve
u2, f2, σ2, F2, α2 = beam2.solve(max_iter=100)

print("Tip deflection (ABAQUS reference):")
print(f"    X: {u2[tip,0].item():.3f} (-0.506)")
print(f"    Y: {u2[tip,1].item():.3f} ( 0.000)")
print(f"    Z: {u2[tip,2].item():.3f} (-2.047)")
print(f"Maximum stress σ_xx: {σ2[:,0,0].max():.3f} (  97.325)")
print(f"Minimum stress σ_xx: {σ2[:,0,0].min():.3f} (-104.776)")

Tip deflection (ABAQUS reference):
    X: -0.759 (-0.506)
    Y: -0.000 ( 0.000)
    Z: -2.351 (-2.047)
Maximum stress σ_xx: 118.633 (  97.325)
Minimum stress σ_xx: -99.316 (-104.776)


## Plot comparison

In [4]:
plotter = pyvista.Plotter()
beam1.plot(color="gray", plotter=plotter, opacity=0.1)
beam1.plot(u=u1, color="cyan", opacity=0.8, plotter=plotter)
beam2.plot(u=u2, color="orange", opacity=0.8, plotter=plotter)
plotter.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…